<a href="https://colab.research.google.com/github/RY0699/DatingApp/blob/main/DatingApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/NetML/Data

/content/drive/MyDrive/NetML/Data


In [ ]:
import pandas as pd

In [ ]:
def read_n_clean(file):
  df = pd.read_csv(file)
  
  uniqueMACs = df['userMAC'].nunique()
  uniqueAPs = df['APNAME'].nunique()
  totalRecords = df['userMAC'].count() 
  # print("Unique Users: ", uniqueMACs)
  # print("Unique Number of Access Points: ", uniqueAPs)
  # print("Total Number of Records: ", totalRecords)

  clean_df1 = df.copy()

  # cleaning all data and keeping only data that is more than 30 min long
  clean_df1 = clean_df1.drop(clean_df1[clean_df1.APNAME == "unknown"].index)
  clean_df1 = clean_df1.drop(clean_df1[clean_df1.APNAME == "AP0007"].index)
  clean_df1 = clean_df1.drop(clean_df1[clean_df1.endTime - clean_df1.startTime == 0].index)
  clean_df1 = clean_df1.drop(clean_df1[clean_df1.endTime - clean_df1.startTime <= 1800].index)

  # eliminating all night time extended times bc that's dorms
  start_time = int(pd.to_datetime('4/7/2012 0:00').timestamp())
  end_time = int(pd.to_datetime('4/7/2012 5:00').timestamp())
  clean_df1 = clean_df1.drop(clean_df1[(clean_df1.startTime >= start_time) & (clean_df1.endTime <= end_time) & (clean_df1.endTime - clean_df1.startTime >= 10800)].index)

  # recordsAfterCleaning = clean_df1['userMAC'].count()
  # percentageOfRecordsRemoved = (1 - (recordsAfterCleaning / totalRecords))
  # print(f"Percentage of Records Removed: {percentageOfRecordsRemoved:.2%}")
  # print(clean_df1['userMAC'].nunique())

  return clean_df1




In [ ]:
#clean each file

df1 = read_n_clean('DHCP_April_2012_devType-day-1.csv')
df2 = read_n_clean('DHCP_April_2012_devType-day-2.csv')
df3 = read_n_clean('DHCP_April_2012_devType-day-3.csv')
df4 = read_n_clean('DHCP_April_2012_devType-day-4.csv')
df5 = read_n_clean('DHCP_April_2012_devType-day-5.csv')
df6 = read_n_clean('DHCP_April_2012_devType-day-6.csv')
df7 = read_n_clean('DHCP_April_2012_devType-day-7.csv')

#define list of dataframe
df_list = []

#append each individual clean dataframe to the list of dataframes
df_list.append(df1)
df_list.append(df2)
df_list.append(df3)
df_list.append(df4)
df_list.append(df5)
df_list.append(df6)
df_list.append(df7)

big_df = pd.concat(df_list)
df = big_df

In [ ]:
# grouping together all locations a particular user visits and keeping track of all AP's a user visits and times for each
grouped_df = df.groupby(df.userMAC)
grouped_lists = grouped_df.apply(lambda x: [list(x['APNAME']), list(x['startTime']), list(x['endTime'])])
grouped_lists = grouped_lists.reset_index()

In [ ]:
# making data more organized and relabeling
grouped_lists.rename(columns = {0:'places'}, inplace = True)
grouped_lists["food"] = 0
grouped_lists["fitness"] = 0
grouped_lists["social"] = 0
grouped_lists["academic"] = 0
grouped_lists["other"] = 0
grouped_lists = grouped_lists[["userMAC", "food", "fitness", "social", "academic", "other", "places"]]
df = grouped_lists
df.index.name = "userMAC"

In [ ]:
# creating a map for prefixes to categories
mappings = {y[0]: y[1:] for y in [x.split(",") for x in open('prefixMap.csv').read().split('\n') if x]}

In [ ]:
import re


# JUST USED FOR TESTING

# curr = df.iloc[0]["places"]
# timeSpentAtAP = curr[2][0] - curr[1][0]

# place = curr[0][0]
# print(place)
# print(curr[0][0], curr[1][0], curr[2][0])
# longest = place[0]

# print(len(place))
# for i in range(len(place)):
#   if place[:i] in mappings:
#     longest = place[:i]

# for i in mappings[longest]:
#   df.at[0, i] = timeSpentAtAP


# for each item in new organized df, got the time spent at each AP and summed it up with the current value for time spent in a category
for index, row in df.iterrows():
    curr = df.iloc[index]["places"]
    for i in range(len(curr[0])):
      timeSpentAtAP = curr[2][i] - curr[1][i]
      place = curr[0][i]
      longest = place[0]
      for i in range(len(place)):
        if place[:i] in mappings:
          longest = place[:i]

      # not considering the APs that have no mapping
      if longest not in mappings:
        continue
    
      for cat in mappings[longest]:
        new = re.sub(r'[^A-Za-z0-9]+', '', cat)
        df.at[index, new] += timeSpentAtAP

In [ ]:
# removing the places category that was created to help restructure df
df = df.drop(["places"], axis = 1).reset_index(drop=True)

In [ ]:
df

,userMAC,food,fitness,social,academic,other
0,0,0,0,0,68273,0
1,19,0,0,0,0,2751
2,26,0,0,0,0,0
3,28,0,0,0,22163,0
4,30,0,0,0,14980,0
...,...,...,...,...,...,...
47124,86325,0,0,0,23542,0
47125,86326,3554,0,0,0,0
47126,86327,0,0,1919,31758,0
47127,86331,0,0,0,7488,0


In [ ]:
def clean_zerotime(df):
  clean_df1 = df.copy()

  # cleaning all data and keeping only data that is more than 30 min long
  clean_df1 = clean_df1.drop(clean_df1[(clean_df1.food ==0) & (clean_df1.fitness==0) & (clean_df1.social ==0) & (clean_df1.academic==0) & (clean_df1.other==0) ].index)

  return clean_df1


df = clean_zerotime(df)

In [ ]:
df

,userMAC,food,fitness,social,academic,other
0,0,0,0,0,68273,0
1,19,0,0,0,0,2751
3,28,0,0,0,22163,0
4,30,0,0,0,14980,0
5,31,0,0,0,239816,0
...,...,...,...,...,...,...
47124,86325,0,0,0,23542,0
47125,86326,3554,0,0,0,0
47126,86327,0,0,1919,31758,0
47127,86331,0,0,0,7488,0


In [ ]:
import numpy as np

# set dating preferences from random selection for each other user

genders = [1, 0]

df['userGender'] = np.random.choice(genders, size=len(df), p=[0.5, 0.5])

dating_preferences = ['smoking', 'drinking', 'seriousRelationship']

for dating_preference in dating_preferences:
  df[dating_preference] = np.random.choice([0, 1], size=len(df), p=[0.5, 0.5])



In [ ]:
# scaling

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

df[['food', 'fitness', 'social', 'academic', 'other']] = scaler.fit_transform(df[['food', 'fitness', 'social', 'academic', 'other']])

In [ ]:
df

,userMAC,food,fitness,social,academic,other,userGender,smoking,drinking,seriousRelationship
0,0,0.000000,0.0,0.000,0.115338,0.000000,0,1,1,0
1,19,0.000000,0.0,0.000,0.000000,0.008547,0,1,1,1
3,28,0.000000,0.0,0.000,0.037442,0.000000,1,1,0,1
4,30,0.000000,0.0,0.000,0.025307,0.000000,0,1,1,0
5,31,0.000000,0.0,0.000,0.405138,0.000000,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...
47124,86325,0.000000,0.0,0.000,0.039771,0.000000,0,0,0,0
47125,86326,0.027482,0.0,0.000,0.000000,0.000000,1,1,0,0
47126,86327,0.000000,0.0,0.004,0.053651,0.000000,0,1,1,1
47127,86331,0.000000,0.0,0.000,0.012650,0.000000,1,1,0,1


In [ ]:
# clustering

from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score

s_scores = []

db_scores = []

for i in range(2, 20):
    
    hac = AgglomerativeClustering(n_clusters=i)
    
    hac.fit(df[['food', 'fitness', 'social', 'academic', 'other']])
    
    cluster_assignments = hac.labels_
    
    s_scores.append(silhouette_score(df[['food', 'fitness', 'social', 'academic', 'other']], cluster_assignments))
    
    db_scores.append(davies_bouldin_score(df[['food', 'fitness', 'social', 'academic', 'other']], cluster_assignments))

In [ ]:
from matplotlib import pyplot as plt

def plot_evaluation(y, x=cluster_cnt):
    """
    Plots the scores of a set evaluation metric. Prints out the max and min values of the evaluation scores.
    """
    
    # Creating a DataFrame for returning the max and min scores for each cluster
    df = pd.DataFrame(columns=['Cluster Score'], index=[i for i in range(2, len(y)+2)])
    df['Cluster Score'] = y
    
    print('Max Value:\nCluster #', df[df['Cluster Score']==df['Cluster Score'].max()])
    print('\nMin Value:\nCluster #', df[df['Cluster Score']==df['Cluster Score'].min()])
    
    # Plotting out the scores based on cluster count
    plt.figure(figsize=(16,6))
    plt.style.use('ggplot')
    plt.plot(x,y)
    plt.xlabel('# of Clusters')
    plt.ylabel('Score')
    plt.show()
    
# Running the function on the list of scores
plot_evaluation(s_scores)

plot_evaluation(db_scores)

In [ ]:
# KMEANS

from sklearn.cluster import KMeans


s_scores = []

db_scores = []

for i in range(2, 20):
    k_means = KMeans(n_clusters=i)
    
    k_means.fit(df[['food', 'fitness', 'social', 'academic', 'other']])
    
    cluster_assignments = k_means.predict(df[['food', 'fitness', 'social', 'academic', 'other']])
 
    s_scores.append(silhouette_score(df[['food', 'fitness', 'social', 'academic', 'other']], cluster_assignments))
    
    db_scores.append(davies_bouldin_score(df[['food', 'fitness', 'social', 'academic', 'other']], cluster_assignments))

In [ ]:
from matplotlib import pyplot as plt

def plot_evaluation(y, x=cluster_cnt):
    """
    Plots the scores of a set evaluation metric. Prints out the max and min values of the evaluation scores.
    """
    
    # Creating a DataFrame for returning the max and min scores for each cluster
    df = pd.DataFrame(columns=['Cluster Score'], index=[i for i in range(2, len(y)+2)])
    df['Cluster Score'] = y
    
    print('Max Value:\nCluster #', df[df['Cluster Score']==df['Cluster Score'].max()])
    print('\nMin Value:\nCluster #', df[df['Cluster Score']==df['Cluster Score'].min()])
    
    # Plotting out the scores based on cluster count
    plt.figure(figsize=(16,6))
    plt.style.use('ggplot')
    plt.plot(x,y)
    plt.xlabel('# of Clusters')
    plt.ylabel('Score')
    plt.show()
    
# Running the function on the list of scores
plot_evaluation(s_scores)

plot_evaluation(db_scores)

In [ ]:
# Instantiating HAC
hac = AgglomerativeClustering(n_clusters=5)

# Fitting
hac.fit(df[['food', 'fitness', 'social', 'academic', 'other']])

# Getting cluster assignments
cluster_assignments = hac.labels_

# Assigning the clusters to each profile
df['Cluster #'] = cluster_assignments

# Viewing the dating profiles with cluster assignments
df

In [ ]:
import random

rand_cluster = random.choice(df['Cluster #'].unique())

# Assigning the Cluster Profiles as a new DF
group = df[df['Cluster #']==rand_cluster].drop('Cluster #', axis=1)

corr_group = group.T.corr()
print(corr_group)


## Finding the Top 10 Similar Users

# Randomly selecting a user
random_user = random.choice(corr_group.index)

print("Top 10 most similar users to User #", random_user, '\n')

# Creating a DF with the top 10 most similar users to the selected user
top_10_sim = corr_group[[random_user]].sort_values(by=[random_user],axis=0, ascending=False)[1:11]

# Printing out the results
print(top_10_sim)

print("\nThe most similar user to User #", random_user, "is User #", top_10_sim.index[0])
